# 유한 오토마타 Finite Automata

오토마타를 "상태 기계"(State Machine)라고도 한다. 따라서 유한 오토마타(Finte Automata; 줄여서 FA)는 "유한 상태 기계"(Finate State Macheime; 줄여서 FSM)라고도 한다. 여기서는 오토마타라는 이름으로 부르겠다.

FA는 라벨된 방향 그래프(labeled directed graph)로 생각하면 된다. 그래프의 노드가 상태에 해당하고 화살표에 알파벳 라벨이 붙어 있으며, 하나의 시작 상태(initial state)가 있고 어러개(0개 이상)의 종료 상태(final state)가 있다. FA는 시작상태에서 화살표를 따라서 여러번 진행하여 종료상태에 도달하기까지 지나온 화살표의 라벨을 순서대로 나열한 문자열(글줄)을 받아들이는(accepts) 기계이다.
FA는 시작상태에서 종료상태로 갈 수 있는 가능한 모든 경로에 해당하는 문자열의 집합, 즉 어떤 언어를 규정한다고 볼 수 있다.

이론적으로 흥미로운 사실은 앞서 배운 정규식(RE)로 정의할 수 있는 언어와
여기서 다루는 유한 오토마타(FA)로 규정할 수 있는 언어의 범위가 정확히 일치한다는 점이다. 즉,
 * 어떤 RE로 정의되는 언어를 규졍하는 FA를 구성할 수 있으며 
 * 어떤 FA로 규정되는 언어를 정의하는 RE를 작성할 수 있다.

실용적인 관점에서는 전자가 더 흥미롭다.
왜내하면 RE는 수학적 표기에 가깝지만 FA는 바로 알고리듬 즉 프로그램으로 작성하기에 알맞은 구조이기 때문이다.
RE는 사람이 읽기 편한 수학적 표기법이며 어떤 성질의 언어인지 그 언어의 집합을 정의하기에 알맞은 구조이다.
하지만 주어진 문자열이 언어에 속하는지 아닌지 검사하는 알고리듬을 어떻게 작성할지에 대해서는
RE의 문법구조가 직접적인 힌트가 되지는 못한다.
반면 FA는 사람이 읽기에는 좀 불친절하지만 시작상태로부터 상태전이 화살표를 따라가면 종료상태에 도달하는지 검사하는 알고리듬으로 옮기기 쉬운 명세이다.
따라서 RE로 정의되는 언어를 규정하는 FA를 자동으로 생성하여 주어진 문자열이
그 FA가 받아들이는 문자열인지 검사함으로써 어떤 문자열이 RE가 표현하는 언어에 속하는지 검사하는 프로그램을 작성할 수 있다.

## DFA, NFA, NFA-$\varepsilon$
보통 형식언어 및 오토마타를 주로 다루는 교재에서는 다음과 같이 네 단계를 거쳐 변환하여 상태가 최소화된 DFA를 얻어낸다.

RE $\xrightarrow{\qquad\qquad}$
NFA-$\varepsilon$ $\xrightarrow{\quad\varepsilon~전이~제거\quad}$
NFA $\xrightarrow{\qquad\qquad}$
DFA $\xrightarrow{\quad상태~최소화\quad}$
DFA

이렇게 하면 상태가 최소화되어 효율적인 정규언어 검사기를 만들 수 있으며
실제로 구문분석기(lexer)를 생성하는 프로그램들이 대개 이러한 방식으로 만들어져 있다.
여기서는 RE와 NFA에 대해서만 다룰 것이므로 DFA나 NFA-$\varepsilon$을 비롯해서 위 그림에 대한
자세한 내용은 형식언어/오토마타/계산이론 관련 교재를 참고하여 시간이 날 때 틈틈이 스스로 공부해 두는 것이 좋다.
참고로 DFA, NFA, NFA-$\varepsilon$ 세 가지 종류의 FA가 규정할 수 있는 언어의 범위가 정확히 일치한다.
그래서 표면적으로 보기에는 더 많은 기능을 가진 NFA-$\varepsilon$로부터 $\varepsilon$-전이 기능을 제외한 NFA로
그리고 비결정성을 제외한 DFA로의 변환이 일반적으로 가능하다.

보통 컴퓨터 관련 전공 학과에서 프로그래밍 언어나 컴파일러 등의 선수과목으로
형식언어/오토마타/계산이론을 다루는 과목이 있는 것이 이론을 탄탄히 익히기에 바람직한데,
아쉽게도 우리 학과에는 따로 없다. 그래서 이 과목 초반부에 최단시간 안에 함수형 프로그래밍과
정규언어 관련 내용을 같이 익혀야 하는 상황이다.
그러니까 단계를 줄여서 RE에서 NFA까지 그냥 한꺼번에 가고
그 다음 단계들 없이, 즉 NFA를 DFA로 변환하지 않고 그냥 NFA 명세를 이용해 바로 실행하는 Haskell 프로그램을 작성해 보도록 하자.

## NFA의 하스켈 타입 정의와 상태 전이 함수

일반적으로 NFA를 이론적으로 설명할 때는 다섯 개의 요소로 이루어진 다음과 같은 순서쌍으로 정의한다. (참고로 DFA나 NFA-$\epsilon$같은 다른 종류의 FA들도 비슷한 방식으로 다섯 순서쌍으로 정의된다.)

$$(Q,\Sigma,\Delta,q_0,F)$$

여기서 각각의 요소가 나타내는 바는 아래와 같다.
 * $Q$는 오토마타가 갖는 상태의 집합이다. 그래프로 표현할 때는 노드의 집합에 해당.
 * $\Sigma$는 라벨로 사용되는 알파벳의 집합이다.
 * $\Delta \subseteq Q \times \Sigma \times Q$는 상태 전이 관계이다. 그래프로 표현할 때는 라벨이 붙은 화살표에 해당한다.
 * $q_0 \in Q$는 초기 상태이다.
 * $F \subseteq Q$는 최종 상태의 집합이다. 그래프로 그릴 때는 보통 원을 이중으로 겹쳐 그린다.

(위 내용을 수강생들에게 확실히 이해시키기 위해서 수업시간에 칠판에 예제를 하나 그려서 설명한다.)

In [13]:
type State = Int  -- 상태 타입은 유한 정수 타입인 Int를 쓰기로 하자
type Label = Char -- 라벨 타입, 즉 알파벳 타입; RE와 마찬가지로 Char를 쓰기로 하자
type Delta = [(State, Label, State)] -- 관계 집합(리스트)로 상태 전이

-- 알파벳을 Char로 정했으므로 따로 표기할 필요가 없어 다섯순서쌍 데신 네순서쌍으로 NFA 정의
type NFA = (State, Delta, State, [State]) -- (상태 최대값, 상태 전이 관계, 초기 상태, 종료 상태 집합)

-- 관계 집합으로부터 상태 전이 함수 정의
delta :: NFA -> (State -> Label -> [State])
delta (_,ds,_,_) =
   \ st lb ->
     [q | (p,l,q) <- ds, p == st, l == lb]

In [14]:
-- 수업 예제
ds = [(0,'a',1),(0,'a',2),(1,'b',2),(2,'a',1)]

[q | (p,l,q) <- ds, p == 0, l == 'a']

[q | (p,l,q) <- ds, p == 1, l == 'a']

[q | (p,l,q) <- ds, p == 1, l == 'b']

[q | (p,l,q) <- ds, p == 2, l == 'b']

[1,2]

[]

[2]

[]

## 정규식이 표현하는 언어를 인식하는 NFA 생성
우선 RegExGen 노트북에서 사용했떤 정규식 데이타 타입과 유틸리티 함수들을 그대로 복사해 오자.

In [15]:
import Data.List (union)

data RE -- 정규식 데이타 타입
  = Empty
  | Epsilon
  | Alphabet Char
  | Concat RE RE
  | Union RE RE
  | Kleene RE
  deriving Show

-- 문자열을 Concat으로 이어진 정규식으로 변환해주는 유틸리티 함수
string2re :: String -> RE
string2re "" = Epsilon
string2re s  = foldr1 Concat (map Alphabet s)

import IHaskell.Display

ppRE r = Display [html(formatRE r)]

formatRE Empty = "∅"
formatRE Epsilon = "ε"
formatRE (Alphabet c) = c:[]
formatRE (Concat r1 r2) = formatRE r1 ++ formatRE r2
formatRE (Union r1 r2) = "(" ++ formatRE r1 ++ "+" ++ formatRE r2 ++ ")"
formatRE (Kleene r) = "(" ++ formatRE r ++ ")*"

In [16]:
string2re "aaaa"

Concat (Alphabet 'a') (Concat (Alphabet 'a') (Concat (Alphabet 'a') (Alphabet 'a')))

In [17]:
ppRE (Union (string2re "aaaa") (Kleene(string2re "bb")))

(aaaa+(bb)*)

In [28]:
-- RE를 NFA로 변환하는 함수 실제 6가지 각각의 경우에 대한 함수가 이후에 따로 작성되어 있다
-- re2nfa는 초기상태가 0인 NFA를 생성하도록 한다.
re2nfa :: RE -> NFA
re2nfa Empty = emptyNFA
re2nfa Epsilon = epsilonNFA
re2nfa (Alphabet c) = alphaNFA c
re2nfa (Concat r1 r2) = concatNFA (re2nfa r1) (re2nfa r2)
re2nfa (Union r1 r2) = unionNFA (re2nfa r1) (re2nfa r2)
re2nfa (Kleene r) = kleeneNFA (re2nfa r)

-- Emtpy 정규식에 해당하는 NFA 생성
emptyNFA :: NFA
emptyNFA = (0,[],0,[])
-- Epsilon 정규식에 해당하는 NFA 생성
epsilonNFA :: NFA
epsilonNFA = (0,[],0,[0])
-- Alphabet c 정규식에 해당하는 NFA 생성
alphaNFA :: Label -> NFA
alphaNFA c = (1,[(0,c,1)],0,[1])

-- 기존의 nfa로 새로운 nfa를 만들어내는 귀납적 경우를 정의하기 위한 도우미 함수
mapNFAstate :: (State -> State) -> NFA -> NFA
mapNFAstate f (m,ds,s,qs) = (f m, [(f p, c, f q) | (p,c,q)<-ds], f s, map f qs)

{-
-- 버그가 있는 unionNFA 정의; 언뜻 생각하면 이러면 될거같지만 버그가 있다
unionNFA :: NFA -> NFA -> NFA
unionNFA nfa1@(m1,ds1,_,qs1) nfa2 = (m2, union ds1 ds2, 0, union qs1 qs2)
  where
  (m2,ds2,_,qs2) = mapNFAstate f nfa2
  f 0 = 0
  f x = x + m1
-}

unionNFA :: NFA -> NFA -> NFA
unionNFA nfa1 nfa2
  = (m2, union ds1' ds2', 0, union qs1' qs2')
  where
  -- nfa1 상태를 1씩 증가
  (m1,ds1,s1,qs1) = mapNFAstate (+1) nfa1
  -- 1만큼 증가된 nfa1과 겹치지 않게 nfa2 상태를 증가
  (m2,ds2,s2,qs2) = mapNFAstate (+(m1+1)) nfa2
  ds1' = [(0,l,q) | (p,l,q)<-ds1, p==s1] `union` ds1 
  ds2' = [(0,l,q) | (p,l,q)<-ds2, p==s2] `union` ds2
  qs1' = if s1 `elem` qs1 then 0:qs1 else qs1
  qs2' = if s2 `elem` qs2 then 0:qs2 else qs2

concatNFA :: NFA -> NFA -> NFA
concatNFA = undefined

kleeneNFA :: NFA -> NFA
kleeneNFA = undefined

In [19]:
complexnumber n = x + y
  where
  x = n ^ 2
  y = n ^ 3

complexnumber' n =
  let
    x = n ^ 2
    y = n ^ 3
   in
    x + y


complexnumber 2
complexnumber' 2

12

12

In [20]:
emptyNFA
epsilonNFA
alphaNFA 'a'
alphaNFA 'b'

(0,[],0,[])

(0,[],0,[0])

(1,[(0,'a',1)],0,[1])

(1,[(0,'b',1)],0,[1])

In [29]:
{-
-- 버그있는 unionNFA를 테스트하던 코드
mapNFAstate (\x -> case x of {0 -> 0; x -> x + 1}) (alphaNFA 'b')
unionNFA (alphaNFA 'a') (alphaNFA 'b')
-}
alphaNFA 'a'

mapNFAstate (+1) (alphaNFA 'a')
mapNFAstate (+10) (alphaNFA 'a')


mapNFAstate (+2) (alphaNFA 'b')

unionNFA (alphaNFA 'a') (alphaNFA 'b')

(1,[(0,'a',1)],0,[1])

(2,[(1,'a',2)],1,[2])

(11,[(10,'a',11)],10,[11])

(3,[(2,'b',3)],2,[3])

(4,[(0,'a',2),(1,'a',2),(0,'b',4),(3,'b',4)],0,[2,4])

## NFA상태를 그래프로 나타내기
그래프로 나타내기 위해서  툴을 사용한다.

https://graphviz.glitch.me/


해당 툴에서는 문자를 유니코드화 하여 URL로 처리하기 때문에 
우선 알파벳과 숫자를 제외한 글자를 유니코드값으로 변환하기 위한 유틸리티 함수를 가져온다.


https://www.rosettacode.org/wiki/URL_encoding#Haskell

In [22]:
import qualified Data.Char as Char
import Text.Printf
 
encode :: Char -> String
encode c
  | c == ' ' = "%20" 
  | Char.isAlphaNum c || c `elem` "-._~" = [c]  
  | c `elem` "\"" = "%22"
  | otherwise = printf "%%%02X" c
 
urlEncode :: String -> String
urlEncode = concatMap encode

초기 상태, 상태 전이 관계, 종료 상태 집합을 인식하여 해당 툴의 형식으로 바꿔줄 함수와 
해당 문자열을 연결하여 이미지 태그로 변환하는 함수를 생성한다.

In [23]:
import IHaskell.Display
-- 초기 상태를 문자열로 변환하는 함수
formatStart :: Int -> String
formatStart p = "start [shape=" ++ show "none" ++ "]; start -> " ++ show p ++ "; "

-- 상태 전이 관계를 문자열로 변환하는 함수
formatDelta :: [(Int, Char, Int)] -> String
formatDelta [] = ""
formatDelta ((p,a,q):ds) = show p ++ " -> " ++ show q ++ " [label=" ++ show (a:"") ++ "]; " ++ formatDelta ds

-- 종료 상태 집합을 문자열로 변환하는 함수
formatFinal :: [Int] -> String
formatFinal [] = ""
formatFinal (q:qs) = show q ++ " [shape=" ++ show "doublecircle" ++ "]; " ++ formatFinal qs

-- 위의 상태집합 함수들을 연결하는 함수
nfa2graph (_,ds,p,qs) = "digraph G "
                     ++ "{ node[shape="++show "circle" ++"]; " 
                     ++    formatStart p
                     ++    formatDelta ds
                     ++    formatFinal qs
                     ++ "}"

-- 문자열을 연결해 html img 코드로 만들어 화면에 출력하는 함수
displayGraph nfa = Display [html ("<img src='" ++ url ++ "'/>")]
  where url = "https://graphviz.glitch.me/graphviz?layout=dot&format=svg&mode=download&graph="
           ++ urlEncode (nfa2graph nfa)

In [26]:
aStar = (0,[(0,'a',0)],0,[0])
bStar = (0,[(0,'b',0)],0,[0])
displayGraph aStar
putStr (nfa2graph aStar)

displayGraph emptyNFA
putStr (nfa2graph emptyNFA)

displayGraph epsilonNFA
putStr (nfa2graph epsilonNFA)

displayGraph (alphaNFA 'a')
putStr (nfa2graph (alphaNFA 'a'))

digraph G { node[shape="circle"]; start [shape="none"]; start -> 0; 0 -> 0 [label="a"]; 0 [shape="doublecircle"]; }

digraph G { node[shape="circle"]; start [shape="none"]; start -> 0; }

digraph G { node[shape="circle"]; start [shape="none"]; start -> 0; 0 [shape="doublecircle"]; }

digraph G { node[shape="circle"]; start [shape="none"]; start -> 0; 0 -> 1 [label="a"]; 1 [shape="doublecircle"]; }

In [27]:
displayGraph (unionNFA (alphaNFA 'a') (alphaNFA 'b'))
putStr (nfa2graph (unionNFA (alphaNFA 'a') (alphaNFA 'b')))

displayGraph (unionNFA aStar bStar)
putStr (nfa2graph (unionNFA aStar bStar))

digraph G { node[shape="circle"]; start [shape="none"]; start -> 0; 0 -> 2 [label="a"]; 1 -> 2 [label="a"]; 0 -> 4 [label="b"]; 3 -> 4 [label="b"]; 2 [shape="doublecircle"]; 4 [shape="doublecircle"]; }

digraph G { node[shape="circle"]; start [shape="none"]; start -> 0; 0 -> 1 [label="a"]; 1 -> 1 [label="a"]; 0 -> 2 [label="b"]; 2 -> 2 [label="b"]; 0 [shape="doublecircle"]; 1 [shape="doublecircle"]; 2 [shape="doublecircle"]; }

In [9]:
accepts :: NFA -> String -> Bool
accepts nfa@(_,_,s,finals) str = any (`elem` finals) (steps s str) 
  where
  step :: State -> Label -> [State]
  step = delta nfa
  steps :: State -> [Label] -> [State]
  steps p []     = return p
  steps p (c:cs) = do q <- step p c
                      steps q cs

In [10]:
nfaUnionAB = unionNFA (alphaNFA 'a') (alphaNFA 'b')

accepts nfaUnionAB ""
accepts nfaUnionAB "a"
accepts nfaUnionAB "b"
accepts nfaUnionAB "c"
accepts nfaUnionAB "aa"
accepts nfaUnionAB "ab"
accepts nfaUnionAB "ba"
accepts nfaUnionAB "bb"

False

True

True

False

False

False

False

False

## 주어진 문자열이 정규식을 만족하는지 검사하는 프로그램

In [11]:
accepts' :: RE -> String -> Bool
accepts' re str = re2nfa re `accepts` str

In [12]:
reUnionAB = Union (Alphabet 'a') (Alphabet 'b') 

accepts' reUnionAB ""
accepts' reUnionAB "a"
accepts' reUnionAB "b"
accepts' reUnionAB "c"
accepts' reUnionAB "aa"
accepts' reUnionAB "ab"
accepts' reUnionAB "ba"
accepts' reUnionAB "bb"

False

True

True

False

False

False

False

False